In [2]:
!python3 --version

Python 3.6.9


In [3]:
!pip3 install -q -U pip
!pip3 install -q numpy
!pip3 install -q pandas
!pip3 install -q ckiptagger
!pip3 install -q tqdm
!pip3 install -q tensorflow==1.14.0
!pip3 install -q ipywidgets

In [11]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS, data_utils
from tqdm.notebook import tqdm

In [12]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [13]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [14]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [15]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [28]:
# data_utils.download_data_gdown("./")
ws = WS('./data/D07data/')
pos = POS('./data/D07data/')

Exception ignored in: <bound method WS.__del__ of <ckiptagger.api.WS object at 0x7f7d9f97e2b0>>
Traceback (most recent call last):
  File "/home/eating/.local/lib/python3.6/site-packages/ckiptagger/api.py", line 65, in __del__
    self.model.sess.close()
AttributeError: 'WS' object has no attribute 'model'


In [37]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    
    word_s = ws([title], sentence_segmentation = True, segment_delimiter_set={'?', '？', '!', '！', '。', ',','，', ';', ':', '、'}) 
    word_p = pos(word_s)
    
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [38]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title], sentence_segmentation = True, segment_delimiter_set = {'?', '？', '!', '！', '。', ',','，', ';', ':', '、'}) 
    word_p = pos(word_s)
    
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [44]:
train_title_cuts[120][0]

('國腳', 'Na')

# Bag of Words (BOW)

In [51]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
count = 0
for title_word in train_title_cuts:
    for word in train_title_cuts[title_word]:
        if word[0] not in word2index:
            word2index[word[0]] = count
            index2word[count] = word[0]
            count += 1
# END YOUR CODE

In [58]:
word2index['許']
len(word2index)

6663

In [59]:
index2word[1520]
len(index2word)

6663

In [60]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    word_len = len(word2index)
    vector = np.zeros(word_len)
    for word in pairs:
        vector[word2index[word[0]]] += 1
    # END YOUR CODE
    return vector

In [61]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 1., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [62]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['芒格', '猛', '武磊', '清華', '瓜迪奧拉', '埃姆雷', '清華大學生', '馬卡報', '費基爾', '吳敬璉', '張大仙', '惠若琪', '萊科寧', '魅族', '凱西', '雷軍', '石川佳純', '老馬', '詹姆斯', '東皇', '楊柳夏', '查理芒格', '宇通', '奧尼爾', '郭懿', '雷霆隊', '馬里奧', '萊萬', '群里', '科爾', '波爾', '淘寶', '李盈瑩', '中金', '瓦基弗', '賈躍亭', '西蒙斯', '維亞利', '美年達', '雪迪龍', '埃米爾岡', '泰拳王西', '昌圖', '小明', '神武', '京新', '董明珠', '三星', '巔峰賽', '羅現', 'S10', '韋德', '阿里巴巴', '華創', '泰倫', '朱婷', '中超', '迪士尼', '布茲德里克', '寶二', '阿木', '丘誠', '郎平', '人民日報', '安德森', '鍾馗', '拉卡拉', '馬龍', '郭煒煒', '勒布朗', '建成', '郭躍', '龐氏', '加威廉', '西游', '王者榮', '劉熾平', '美宇', '梅西', '嬴政', '姆巴佩', '巴爾韋德', '特里', '蘇群', '劉歡', '億航', '騎士隊', '科斯塔', '東決', '小白', '哈希', '詹黑皮爾斯', '林美景', '海因克斯 萊萬多夫斯基', '李大霄', '韓寒', '李白', '易', '新游', '力哥']
Na : ['小麥', '空投里', '空氣', '利好', '敵人', '隊員', '狂魔', '中藥', '王朝', '裙子', '雕像', '離子', '費用', '白鰱', '溶洞', '時節', '衣服', '銀牌', '化肥會', '杜鵑花', '麥塊', '米', '城市', '手機', '小額', '網商貸', '小牛', '頻道', '墒情', '軟件', '珍品', '心骨', '乒乓球', '開關', '紅薯苗', '運氣', '發酵劑', '嘲諷度', '抉擇', '皇家', '概要', '探險隊', '美景', '國人', '主義', '良心', '魔王', '分文', '人物', '品種', '中單', '程度', '

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [63]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
        # YOUR CODE HERE
        'DE', 'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY', 'SPCHANGECATEGORY',
        'WHITESPACE'
        # END YOUR CODE
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [66]:
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    dot = sum(a*b for a, b in zip(bow1, bow2))
    norm_a = sum(a*a for a in bow1) ** 0.5
    norm_b = sum(b*b for b in bow2) ** 0.5
    # END YOUR CODE
    return dot/(norm_a*norm_b)

In [67]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [68]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [69]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [70]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [ ]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [ ]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)